In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"beckoliver","key":"556415c0977bbabf21459dcd522412f9"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d jessicali9530/celeba-dataset

100% 1.33G/1.33G [00:34<00:00, 43.2MB/s]
100% 1.33G/1.33G [00:34<00:00, 41.4MB/s]


In [36]:
! kaggle datasets download -d nickj26/places2-mit-dataset

100% 27.8G/27.8G [18:05<00:00, 39.9MB/s]
100% 27.8G/27.8G [18:05<00:00, 27.5MB/s]


In [ ]:
! unzip celeba-dataset.zip

In [37]:
! unzip places2-mit-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: val_256/val_256/Places365_val_00031501.jpg  
  inflating: val_256/val_256/Places365_val_00031502.jpg  
  inflating: val_256/val_256/Places365_val_00031503.jpg  
  inflating: val_256/val_256/Places365_val_00031504.jpg  
  inflating: val_256/val_256/Places365_val_00031505.jpg  
  inflating: val_256/val_256/Places365_val_00031506.jpg  
  inflating: val_256/val_256/Places365_val_00031507.jpg  
  inflating: val_256/val_256/Places365_val_00031508.jpg  
  inflating: val_256/val_256/Places365_val_00031509.jpg  
  inflating: val_256/val_256/Places365_val_00031510.jpg  
  inflating: val_256/val_256/Places365_val_00031511.jpg  
  inflating: val_256/val_256/Places365_val_00031512.jpg  
  inflating: val_256/val_256/Places365_val_00031513.jpg  
  inflating: val_256/val_256/Places365_val_00031514.jpg  
  inflating: val_256/val_256/Places365_val_00031515.jpg  
  inflating: val_256/val_256/Places365_val_00031516.jpg  
  inflating: val_256/

In [ ]:
import numpy as np
import pandas as pd
import os, math, sys
import glob, itertools
import argparse, random
import cv2

from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

random.seed(42)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models import vgg19
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image, make_grid

In [ ]:
n_epochs = 1
batch_size = 16
dataset_name = "/content/img_align_celeba/img_align_celeba"
lr = 0.00008
b1 = 0.5
b2 = 0.999
n_cpu = 4
img_size = 128
mask_size = 64
channels = 3
sample_interval = 500


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, img_size=128, mask_size=64, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.img_size = img_size
        self.mask_size = mask_size
        self.mode = mode
        self.files = sorted(glob.glob("%s/*.jpg" % root))
        self.files = self.files[:-4000] if mode == "train" else self.files[-4000:]

    def apply_random_mask(self, img):
        """Randomly masks image"""
        y1, x1 = np.random.randint(0, self.img_size - self.mask_size, 2)
        y2, x2 = y1 + self.mask_size, x1 + self.mask_size
        masked_part = img[:, y1:y2, x1:x2]
        masked_img = img.clone()
        masked_img[:, y1:y2, x1:x2] = 1

        return masked_img, masked_part

    def apply_center_mask(self, img):
        """Mask center part of image"""
        # Get upper-left pixel coordinate
        i = (self.img_size - self.mask_size) // 2
        masked_img = img.clone()
        masked_img[:, i : i + self.mask_size, i : i + self.mask_size] = 1

        return masked_img, i

    def __getitem__(self, index):

        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        if self.mode == "train":
            # For training data perform random mask
            masked_img, aux = self.apply_random_mask(img)
        else:
            # For test data mask the center of the image
            masked_img, aux = self.apply_center_mask(img)

        return img, masked_img, aux

    def __len__(self):
        return len(self.files)

In [ ]:
transforms_ = [
    transforms.Resize((img_size, img_size), Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ImageDataset(dataset_name, transforms_=transforms_),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)
test_dataloader = DataLoader(
    ImageDataset(dataset_name, transforms_=transforms_, mode="val"),
    batch_size=12,
    shuffle=True,
    num_workers=1,
)

In [ ]:
img, masked_imgs, aux = next(iter(dataloader))
img.shape, masked_imgs.shape, aux.shape

(torch.Size([16, 3, 128, 128]),
 torch.Size([16, 3, 128, 128]),
 torch.Size([16, 3, 64, 64]))

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
def vis(idx):
    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15,10)
    a = img[idx].cpu().detach().numpy()
    a = np.transpose(a, (1, 2, 0))
    ax[0].imshow(a)
    ax[0].set_title("The Original Image")
    ax[1].imshow(np.transpose(masked_imgs[idx], (1, 2, 0)))
    ax[1].set_title("The masked Image")
    ax[2].imshow(np.transpose(aux[idx], (1, 2, 0)))
    ax[2].set_title("Mask Image")

In [ ]:
vis(0)

#SE GAN

In [ ]:
class Generator(nn.Module):
    def __init__(self, channels=3):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),
            nn.BatchNorm2d(512, 0.8),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 4000, 1),

            nn.ConvTranspose2d(4000, 512, 4, stride=2, padding=1), #upsample
            nn.BatchNorm2d(512, 0.8),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1), #upsample
            nn.BatchNorm2d(256, 0.8),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1), #upsample
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1), #upsample
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(),
            nn.Conv2d(64, channels, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 3, stride=1, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, 3, 1, 1)
        )


    def forward(self, img):
        return self.model(img)

In [ ]:
cuda = True if torch.cuda.is_available() else False
os.makedirs("images", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)


patch_h, patch_w = int(mask_size / 2 ** 3), int(mask_size / 2 ** 3)
patch = (1, patch_h, patch_w)

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

def save_sample(batches_done):
    samples, masked_samples, i = next(iter(test_dataloader))
    samples = Variable(samples.type(Tensor))
    masked_samples = Variable(masked_samples.type(Tensor))
    i = i[0].item()  # Upper-left coordinate of mask
    # Generate inpainted image
    gen_mask = generator(masked_samples)
    filled_samples = masked_samples.clone()
    filled_samples[:, :, i : i + mask_size, i : i + mask_size] = gen_mask
    # Save sample
    sample = torch.cat((masked_samples.data, filled_samples.data, samples.data), -2)
    save_image(sample, "images/%d.png" % batches_done, nrow=6, normalize=True)


# Loss function
adversarial_loss = torch.nn.MSELoss()
pixelwise_loss = torch.nn.L1Loss()

# Initialize generator and discriminator
generator = Generator(channels=channels)
discriminator = Discriminator(channels=channels)

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
print(generator)

Generator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.2)
    (14): Conv2d(512, 4000, kernel_si

In [ ]:
print(discriminator)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(512, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


In [ ]:
os.makedirs('/content/saved_models', exist_ok=True)

In [ ]:
gen_adv_losses, gen_pixel_losses, disc_losses, counter = [], [], [], []

for epoch in range(20):

    ### Training ###
    gen_adv_loss, gen_pixel_loss, disc_loss = 0, 0, 0
    tqdm_bar = tqdm(dataloader, desc=f'Training Epoch {epoch} ', total=int(len(dataloader)))
    for i, (imgs, masked_imgs, masked_parts) in enumerate(tqdm_bar):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], *patch).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], *patch).fill_(0.0), requires_grad=False)

        # Configure input
        imgs = Variable(imgs.type(Tensor))
        masked_imgs = Variable(masked_imgs.type(Tensor))
        masked_parts = Variable(masked_parts.type(Tensor))

        ## Train Generator ##
        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_parts = generator(masked_imgs)

        # Adversarial and pixelwise loss
        g_adv = adversarial_loss(discriminator(gen_parts), valid)
        g_pixel = pixelwise_loss(gen_parts, masked_parts)
        # Total loss
        g_loss = 0.001 * g_adv + 0.999 * g_pixel

        g_loss.backward()
        optimizer_G.step()

        ## Train Discriminator ##
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(masked_parts), valid)
        fake_loss = adversarial_loss(discriminator(gen_parts.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        gen_adv_loss, gen_pixel_loss, disc_loss
        gen_adv_losses, gen_pixel_losses, disc_losses, counter

        gen_adv_loss += g_adv.item()
        gen_pixel_loss += g_pixel.item()
        gen_adv_losses.append(g_adv.item())
        gen_pixel_losses.append(g_pixel.item())
        disc_loss += d_loss.item()
        disc_losses.append(d_loss.item())
        counter.append(i*batch_size + imgs.size(0) + epoch*len(dataloader.dataset))
        tqdm_bar.set_postfix(gen_adv_loss=gen_adv_loss/(i+1), gen_pixel_loss=gen_pixel_loss/(i+1), disc_loss=disc_loss/(i+1))

        # Generate sample at sample interval
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_sample(batches_done)

    torch.save(generator.state_dict(), "/content/saved_models/generator.pth")
    torch.save(discriminator.state_dict(), "/content/saved_models/discriminator.pth")

Training Epoch 0 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 1 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 2 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 3 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 4 :   0%|          | 0/12413 [00:00<?, ?it/s]

...

Training Epoch 15 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 16 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 17 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 18 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 19 :   0%|          | 0/12413 [00:00<?, ?it/s]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=counter,
    y=gen_adv_losses,
    mode='lines+markers',  # Adding markers to the line
    name='Gen Adv Loss',
    line=dict(color='RoyalBlue', dash='dash')  # Changing line color to RoyalBlue and making it dashed
))

fig.update_layout(
    width=1000,
    height=500,
    title="Generator Adversarial Loss",
    xaxis_title="Number of training examples seen",
    yaxis_title="Gen Adversarial Loss (MSELoss)",
    plot_bgcolor='rgba(0,0,0,0)',  # Makes the background transparent
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightPink'),  # Adding a grid for the x-axis
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightPink'),  # Adding a grid for the y-axis
    font=dict(family="Courier New, monospace", size=18, color="#7f7f7f")  # Changing font settings
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=counter,
    y=gen_pixel_losses,
    mode='lines+markers',  # Adding markers to the line for more detail
    name='Gen Pixel Loss',
    line=dict(color='DarkSlateGrey', width=2, dash='dot'),  # Changing line color, width, and style
    marker=dict(color='Crimson', size=8, symbol='circle')  # Customizing markers
))

fig.update_layout(
    width=1000,
    height=500,
    title={
        'text': "Generator Pixel Loss Over Training",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},  # Centering the title
    title_font=dict(size=25, family='Arial', color='RebeccaPurple'),  # Customizing title font
    paper_bgcolor='Lavender',  # Changing the background color of the plot
    plot_bgcolor='AliceBlue',  # Changing the plotting area background color
    xaxis_title="Number of Training Examples Seen",
    yaxis_title="Generator Pixel Loss (L1 Loss)",
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='Black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='Navy')),  # Customizing the x-axis
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='LightPink',
        linecolor='Black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='Navy')),  # Customizing the y-axis
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )  # Positioning the legend
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=counter, y=disc_losses, mode='lines', name='Discriminator Loss', marker_color='seagreen'))

fig.update_layout(
    width=1000,
    height=500,
    title="Discriminator Loss",
    xaxis_title="Number of training examples seen",
    yaxis_title="Disc Loss (MSELoss)"),
fig.show()

Output hidden; open in https://colab.research.google.com to view.

#DSE GAN

In [ ]:
class MGenerator(nn.Module):
    def __init__(self, channels=3):
        super(MGenerator, self).__init__()

        def downsample(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2))
            return layers

        def upsample(in_feat, out_feat, normalize=True):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.ReLU())
            return layers

        self.model = nn.Sequential(
            *downsample(channels, 64, normalize=False),
            *downsample(64, 64),
            *downsample(64, 128),
            *downsample(128, 256),
            *downsample(256, 512),
            nn.Conv2d(512, 4000, 1),
            *upsample(4000, 512),
            *upsample(512, 256),
            *upsample(256, 128),
            *upsample(128, 64),
            nn.Conv2d(64, channels, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


class MDiscriminator(nn.Module):
    def __init__(self, channels=3):
        super(MDiscriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, stride, normalize):
            """Returns layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 3, stride, 1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = channels
        for out_filters, stride, normalize in [(64, 2, False), (128, 2, True), (256, 2, True), (512, 1, True)]:
            layers.extend(discriminator_block(in_filters, out_filters, stride, normalize))
            in_filters = out_filters

        layers.append(nn.Conv2d(out_filters, 1, 3, 1, 1))

        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)

In [ ]:
load_pretrained_models = True
madversarial_loss = torch.nn.MSELoss()
mpixelwise_loss = torch.nn.L1Loss()

# Initialize generator and discriminator
mgenerator = MGenerator(channels=channels)
mdiscriminator = MDiscriminator(channels=channels)

# Load pretrained models
if load_pretrained_models:
    mgenerator.load_state_dict(torch.load("/content/saved_models/generator.pth"))
    mdiscriminator.load_state_dict(torch.load("/content/saved_models/discriminator.pth"))

if cuda:
    mgenerator.cuda()
    mdiscriminator.cuda()
    madversarial_loss.cuda()
    mpixelwise_loss.cuda()

# Initialize weights
mgenerator.apply(weights_init_normal)
mdiscriminator.apply(weights_init_normal)

# Optimizers
moptimizer_G = torch.optim.Adam(mgenerator.parameters(), lr=lr, betas=(b1, b2))
moptimizer_D = torch.optim.Adam(mdiscriminator.parameters(), lr=lr, betas=(b1, b2))

mTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [32]:
n_epochs = 10

mgen_adv_losses, mgen_pixel_losses, mdisc_losses, mcounter = [], [], [], []

for epoch in range(n_epochs):

    ### Training ###
    gen_adv_loss, gen_pixel_loss, disc_loss = 0, 0, 0
    tqdm_bar = tqdm(dataloader, desc=f'Training Epoch {epoch} ', total=int(len(dataloader)))
    for i, (imgs, masked_imgs, masked_parts) in enumerate(tqdm_bar):

        # Adversarial ground truths
        valid = Variable(mTensor(imgs.shape[0], *patch).fill_(1.0), requires_grad=False)
        fake = Variable(mTensor(imgs.shape[0], *patch).fill_(0.0), requires_grad=False)

        # Configure input
        imgs = Variable(imgs.type(mTensor))
        masked_imgs = Variable(masked_imgs.type(mTensor))
        masked_parts = Variable(masked_parts.type(mTensor))

        ## Train Generator ##
        moptimizer_G.zero_grad()

        # Generate a batch of images
        gen_parts = mgenerator(masked_imgs)

        # Adversarial and pixelwise loss
        g_adv = madversarial_loss(mdiscriminator(gen_parts), valid)
        g_pixel = mpixelwise_loss(gen_parts, masked_parts)
        # Total loss
        g_loss = 0.001 * g_adv + 0.999 * g_pixel

        g_loss.backward()
        moptimizer_G.step()

        ## Train Discriminator ##
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = madversarial_loss(mdiscriminator(masked_parts), valid)
        fake_loss = madversarial_loss(mdiscriminator(gen_parts.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        moptimizer_D.step()

        gen_adv_loss, gen_pixel_loss, disc_loss
        mgen_adv_losses, mgen_pixel_losses, mdisc_losses, mcounter

        gen_adv_loss += g_adv.item()
        gen_pixel_loss += g_pixel.item()
        mgen_adv_losses.append(g_adv.item())
        mgen_pixel_losses.append(g_pixel.item())
        disc_loss += d_loss.item()
        mdisc_losses.append(d_loss.item())
        mcounter.append(i*batch_size + imgs.size(0) + epoch*len(dataloader.dataset))
        tqdm_bar.set_postfix(gen_adv_loss=gen_adv_loss/(i+1), gen_pixel_loss=gen_pixel_loss/(i+1), disc_loss=disc_loss/(i+1))

        # Generate sample at sample interval
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_sample(batches_done)

    torch.save(mgenerator.state_dict(), "saved_models/mgenerator.pth")
    torch.save(mdiscriminator.state_dict(), "saved_models/mdiscriminator.pth")

Training Epoch 0 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 1 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 2 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 3 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 4 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 5 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 6 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 7 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 8 :   0%|          | 0/12413 [00:00<?, ?it/s]

Training Epoch 9 :   0%|          | 0/12413 [00:00<?, ?it/s]

In [33]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mcounter,
    y=mgen_adv_losses,
    mode='lines+markers',  # Adding markers to the line
    name='Gen Adv Loss',
    line=dict(color='RoyalBlue', dash='dash')  # Changing line color to RoyalBlue and making it dashed
))

fig.update_layout(
    width=1000,
    height=500,
    title="Generator Adversarial Loss",
    xaxis_title="Number of training examples seen",
    yaxis_title="Gen Adversarial Loss (MSELoss)",
    plot_bgcolor='rgba(0,0,0,0)',  # Makes the background transparent
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightPink'),  # Adding a grid for the x-axis
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightPink'),  # Adding a grid for the y-axis
    font=dict(family="Courier New, monospace", size=18, color="#7f7f7f")  # Changing font settings
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [34]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mcounter,
    y=mgen_pixel_losses,
    mode='lines+markers',  # Adding markers to the line for more detail
    name='Gen Pixel Loss',
    line=dict(color='DarkSlateGrey', width=2, dash='dot'),  # Changing line color, width, and style
    marker=dict(color='Crimson', size=8, symbol='circle')  # Customizing markers
))

fig.update_layout(
    width=1000,
    height=500,
    title={
        'text': "Generator Pixel Loss Over Training",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},  # Centering the title
    title_font=dict(size=25, family='Arial', color='RebeccaPurple'),  # Customizing title font
    paper_bgcolor='Lavender',  # Changing the background color of the plot
    plot_bgcolor='AliceBlue',  # Changing the plotting area background color
    xaxis_title="Number of Training Examples Seen",
    yaxis_title="Generator Pixel Loss (L1 Loss)",
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='Black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='Navy')),  # Customizing the x-axis
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        gridcolor='LightPink',
        linecolor='Black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='Navy')),  # Customizing the y-axis
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )  # Positioning the legend
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [35]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=mcounter, y=mdisc_losses, mode='lines', name='Discriminator Loss', marker_color='seagreen'))

fig.update_layout(
    width=1000,
    height=500,
    title="Discriminator Loss",
    xaxis_title="Number of training examples seen",
    yaxis_title="Disc Loss (MSELoss)"),
fig.show()

Output hidden; open in https://colab.research.google.com to view.